# Milestone 4

#### Objective:

 - Audit a model for bias when the protected class is not available.

#### Workflow:

 - Introduction
 - Set up problem
 - Load data and display primary and secondary sets
 - Load model to audit for bias
 - Train model from proxy variables to predict the protected class
 - Assess the model bias
 - Report your findings

#### Importance to Project:

 - Very often, protected class information is not available when you are auditing for bias or removing bias.  
 - In these cases, the task becomes much more challenging. 
 - You might not have access to the protected class information either because you can only see the model itself and not the underlying data, or because even the original creators don't know the protected class status of each row. 
 - While auditing or mitigating bias without direct access to the protected class introduces a level of uncertainty, we can still reach some conclusions about biased models, but must be open about the uncertainty.
 - The most powerful techniques are beyond the scope of this milestone. However, knowing some baseline techniques can be helpful in easy cases and can help guide a decision whether to pursue a more in-depth audit.

#### Resources:

 - You should be able to proceed with this milestone without any additional resources. However, the following resources will provide useful context and more information.
  - https://arxiv.org/pdf/1906.00285.pdf and accompanying video https://www.youtube.com/watch?v=z-ZI1VoA-O0]. Milestone 4 tracks this paper closely - the main difference is in methodology.
  - https://arxiv.org/pdf/1811.11154.pdf - This paper demonstrates some of the weaknesses of proxy-based approaches
  - https://github.com/cfpb/proxy-methodology - This repository contains the BISG proxy methodology
  - https://amstat.tandfonline.com/doi/full/10.1080/2330443X.2018.1427012 - This paper explains the BISG proxy methodology

## Introduction

We can still assess algorithmic fairness when the protected class is unobserved, provided we have some way to deduce the protected class from the available data. The most famous real-world example of this is the BISG proxy, which uses a Naive Bayes model to predict race membership based on name and zipcode. As [Chen et al](https://arxiv.org/pdf/1811.11154.pdf) note, "This methodology notably supported a $98 million fine against a major auto loan lender".

While the BISG proxy methodology is still used and developed, recent scholarship in machine fairness demonstrates some serious shortcomings and some new directions for modelling without access to protected class data. The fundamental issue is that proxy models, unless highly accurate, could skew the audit. For instance, [Chen et al](https://arxiv.org/pdf/1811.11154.pdf) showed the BISG methodology has been shown to overestimate true disparities. In summary, we need to exercise great care when building proxy models and imputing protected class data.

In terms of new directions for assessing algorithmic fairness when the protected class is unobserved, an interesting paper released this year entitled ["Assessing Algorithmic Fairness with Unobserved Protected Class Using Data Combination"](https://arxiv.org/pdf/1906.00285.pdf) describes a methodology for estimating confidence intervals for bias metrics. While the methodology is too complex to review here, in the course of this milestone it will become clear why confidence intervals are so useful. It is well worth considering for any bias audit where the protected class is unavailable.

Regardless of the chosen methodology, the general setup requires the following datasets:
 - a primary dataset, which contains input variables to the model and the model's final prediction
 - a secondary dataset, which contains some proxy variables which overlap with the primary dataset, and does contain the protected class labels
     
An example of a primary set and secondary set in a lending context with potential racial bias can be found below (proxy variables are surname and zipcode, as in BISG, and they can be found in both the primary and secondary sets)

## Setting up the problem

In our example, you are asked by the WHO to evaluate bias in a model which determines personalized warfarin dosage. A similar model built on the same dataset has already been audited using state-of-the-art partial identification sets in a recent paper by Kallus, Mao and Zhou. We will be using a simpler methodology.

Warfarin is the world's most common anticoagulant, but the correct dosage for the desired effect changes significantly from person to person. Too little warfarin will fail to produce a coagulant effect, while too much warfarin can cause dangerous bleeding, stroke and death. 

In these circumstances, dosage by trial and error is dangerous, so the medical community has built models to predict the ideal warfarin dosage. These models can be found in section 8.2 of ["Assessing Algorithmic Fairness with Unobserved Protected Class Using Data Combination"](https://arxiv.org/pdf/1906.00285.pdf). The makers of the models note that genetic factors correlated with race are predictive of the outcome, so we can expect dosage and dosage prediction to differ by race. In this scenario, the WTO has asked you to audit the model's bias with respect to race. 

For this audit, we are going to simulate an environment where race data is not available. To audit the model's bias with respect to race, one method is to train a proxy model on a primary set as described above. Instead of approaching the problem blindly, you are given two categories of columns: genetic features, and medication features. 

## Load libraries and dataset

 - The following code will load all necessary libraries and data. 
 - The code should load:
   - A primary and secondary dataset
   - An initial model
   - A set of genotype columns and a set of medication columns, which can be each used to train a proxy
   - Columns initially used to train the model
   - A true positive rate helper function

In [ ]:
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
%run "./Milestone 4 declarations.ipynb"

In [ ]:
print('Primary dataset')
display(primary)
print('Secondary dataset')
display(secondary)
print('Confirm model loaded')
print('Genotype columns:\n', genotype_columns,'\n\n')
print('Medication columns:\n', medication_columns,'\n\n')
print('Model training columns:\n', training_columns,'\n\n')
print(help(tpr))
model

## Given the supplied training columns, use the model to predict warfarin dosage for the primary set

 - If loaded properly, you can just use model.predict on the training columns of the primary dataset

## Train a proxy model on the secondary set to impute the missing protected class data

 - The overall goal of this section is to train some proxy models to predict race.
 - The only requirement for a proxy model is that its inputs are available in both primary and secondary sets, and that it predicts the column we are trying to impute (race). A simple Random Forest or LightGBM classifier, trained on the secondary set, will suffice. **Make sure you hold out a test portion of the secondary set to test performance.**
 - Based on the column lists approved by domain experts (medication_columns, genotype_columns), three models are possible: one model uses medication_columns to predict race, one uses genotype_columns to predict race, and one uses both.
 - Create a confusion matrix showing each proxy model's performance on the same test set
 - Conclude which model has the best performance. We will use it going forward.

## Add a race column to the primary dataset by using the proxy model with the best performance

Since the proxy columns are present in the primary and secondary sets, you can now predict the race of each row in the primary dataset. Simply run proxy_model.predict on the column set you chose in the primary set.






##  Assess the model bias with the best proxy

In a real-world context, we will usually have some idea of what specific bias metrics we want to measure and report back on. In this case, we can take some guidance from ["Assessing Algorithmic Fairness with Unobserved Protected Class Using Data Combination"](https://arxiv.org/pdf/1906.00285.pdf) paper and look to the bias metrics they investigated for this dataset. The authors in that case study were interested in the distribution of high dosage recommendations by race ("demographic disparity", see section 8.2 of ["Assessing Algorithmic Fairness with Unobserved Protected Class Using Data Combination"](https://arxiv.org/pdf/1906.00285.pdf)), and the distribution of true positive rates by race ("equal opportunity difference").

 - Grouping the dataset by race, observe how the high dosage model's recommendations are distributed, and how they compare to the ground truth ('target'). Is there a disparity in dosage recommendations by race (**disparate impact**)?
 - Using the tpr helper function, compare the true positive rates for each race. Is there a disparity in TPRs (**equal opportunity difference**)? 

## Report on intermediate findings

 - Were the high dosage recommendation rates roughly equivalent across races, or was there a substantial difference indicating disparate impact?
 - Were the true positive rates roughly equivalent across races, or was there a substantial difference indicating equal opportunity difference?
 - What are some reasons to be cautious about true positive rates calculated with the help of a proxy model?
 - What actions could be taken by data scientists or clinicians as a result of your findings?

##  Assess the model bias using race imputation from other proxies
 - Since we have other proxy models available, we can gain a little more confidence in our findings by seeing consistent results when other proxies are used.
 - Repeat your investigation for high dosage recommendation rate disparities using race imputation from the other proxies. Confirm whether the insights align.
 - Repeat your investigation for TPR disparities using race imputation from the other proxies. Confirm whether the insights align.

## Complete the deliverable

Produce a memo which answers the following questions, taking into account the purpose of the model (assessing the risk of cardiovascular disease)

 - Is there a disparity in true positive rates along racial lines?
 - How could this disparity constitute unwanted bias? What are some possible real-world effects?
 - What are some reasons to be cautious about true positive rates calculated with the help of a proxy model?
 - What actions could be taken by data scientists or clinicians as a result of your findings?

## Reflect on the importance of having a highly accurate proxy when modelling without access to protected class data